In [2]:
import os 
import sys
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
names2=os.listdir('./results/global_evaluation_bis/')
names1=os.listdir('./results/global_evaluation/')

In [70]:
count=0
for name in names2: 
    if name in names1 : 
        count+=1
        dico1=json.load(open('./results/global_evaluation/'+name))
        dico2=json.load(open('./results/global_evaluation_bis/'+name))
        if [len(dico1[key]) for key in dico1.keys()] != [len(dico2[key]) for key in dico2.keys()] : 
            dico1=json.load(open('./results/global_evaluation/'+name))
            dico2=json.load(open('./results/global_evaluation_bis/'+name))
            for i in range(len(list(dico1.keys()))):
                key = list(dico1.keys())[i]
                if len(dico1[key])!=len(dico2[key]) :
                    list1=dico1[key]
                    list2=dico2[key]
                    index=list2.index(np.inf)
                    try : 
                        if (list1[index+1] -  list2[index+2]) > 0.001 : 
                            print(list1[index+1],list2[index+2])
                            print('last')
                    except : 
                        pass
count

25.175683718376362 25.09088452703289
last


64

In [63]:
dico1=json.load(open('./results/global_evaluation/evaluation_with_WIS_of_3D_models_on_pandemic_0_29_and_reach_=14.json'))
dico2=json.load(open('./results/global_evaluation_bis/evaluation_with_WIS_of_3D_models_on_pandemic_0_29_and_reach_=14.json'))

In [64]:
for i in range(len(list(dico1.keys()))):
    key = list(dico1.keys())[i]
    print(key)
    if len(dico1[key])!=len(dico2[key]) :
        list1=dico1[key]
        list2=dico2[key]
        index=list2.index(np.inf)
        if (list1[index+1] -  list2[index+2]) > 0.001 : 
            print(name)

[20]
evaluation_with_WIS_of_3D_models_on_pandemic_0_29_and_reach_=14.json
[40]
[60]
[80]
[100]


IndexError: list index out of range

In [66]:
dico1['[100]']

[19.28366274454855,
 4.9595299073104,
 109.01152427942831,
 44.89718711561237,
 inf]

In [65]:
dico2['[100]']

[19.28366274454855,
 4.9595299073104,
 108.30748187455026,
 44.89718711561237,
 inf,
 2659.6492449779516]

In [55]:
[len(dico1[key]) for key in dico1.keys()]

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]

In [56]:
[len(dico2[key]) for key in dico2.keys()]

[6, 5, 5, 5, 6, 6, 5, 5, 5, 5, 6, 5, 5, 5]

In [30]:
dico1=json.load(open('./results/global_evaluation/evaluation_with_RMSE_of_1D_models_on_pandemic_1_56_and_reach_=14.json'))
dico2=json.load(open('./results/global_evaluation_bis/evaluation_with_RMSE_of_1D_models_on_pandemic_1_56_and_reach_=14.json'))

In [39]:
dico2['[40]'].index(np.inf)
dico2['[40]']

[134.47342316118494,
 157.15727080617887,
 184.57142857142856,
 3.8000117282869894,
 208.62806814940024,
 4.335793311038259,
 inf,
 3541.523132395607,
 177.55635197751565,
 2.294113566538215]

In [42]:
dico1['[40]']

[134.47342316118494,
 157.15727080617887,
 184.57142857142856,
 3.8000117282869894,
 208.62806814940024,
 4.335793311038259,
 inf,
 177.55635197751565,
 2.294113566538215]

In [35]:
dico1['[40]']

[134.47342316118494,
 157.15727080617887,
 184.57142857142856,
 3.8000117282869894,
 208.62806814940024,
 4.335793311038259,
 inf,
 177.55635197751565,
 2.294113566538215]

In [25]:
count=0
for name in names2: 
    thisname=True
    dico = json.load(open('./results/global_evaluation_bis/'+name))
    for key in dico.keys(): 
        if len(dico[key]) != 5 and len(dico[key]) != 9  : 
            thisname=False
    if not thisname : 
        count+=1

count

992

In [9]:
names1[0]

'evaluation_with_RMSE_of_1D_models_on_pandemic_0_0_and_reach_=14.json'

In [13]:
dicotest=json.load(open('./results/global_evaluation_bis/'+'evaluation_with_RMSE_of_3D_models_on_pandemic_0_10_and_reach_=14.json'))
dicotest

{'[20]': [1.413168584642551,
  inf,
  inf,
  5.285714285714286,
  0.18268177975863686,
  2.5967502168151295],
 '[40]': [10.393103219137053,
  16.452259950969953,
  16.57142857142857,
  9.233944248738183,
  13.994509894153556],
 '[60]': [0.3247170199723328,
  17.085830498834184,
  7.857142857142858,
  1.5874418812889317,
  3.366352117644727],
 '[80]': [2.9677324235914497,
  8.05793556092516,
  13.42857142857143,
  15.860532018314117,
  4.664744673829432],
 '[100]': [20.759467412102595,
  15.648772856635333,
  57.857142857142854,
  20.32439413118155,
  inf,
  45.581492168796814],
 '[120]': [42.845094178873865,
  94.90723571291085,
  95.0,
  19.802584283240265,
  47.26744556368328],
 '[140]': [0.984209181847632,
  29.82974828497555,
  135.0,
  80.68755859950795,
  7.769652361671035],
 '[160]': [35.418114412879845,
  229.3009217895717,
  164.0,
  88.61636986649944,
  35.11633178475529],
 '[180]': [21.651764423740588,
  162.78652261946695,
  263.57142857142856,
  122.34221504804191,
  91.27